In [2]:
import numpy as np # linear algebra
import pandas as pd #
import glob
from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [3]:

img_size = (224, 224)
img_array_list = []
cls_list = []

train_dir = 'input/1056lab-defect-detection/train/class'
train_def_dir ='input/DataAugmentation/Class'
for i in range(6):
    x = str(i + 1)
    img_list1 = glob.glob(train_dir + x + '/*.png')
    for i in img_list1:
        img = load_img(i, color_mode='grayscale', target_size=(img_size))
        img_array = img_to_array(img) / 255
        img_array_list.append(img_array)
        cls_list.append(0)

    img_list1 = glob.glob(train_def_dir + x + '_def/*.png')
    for i in img_list1:
        img = load_img(i, color_mode='grayscale', target_size=(img_size))
        img_array = img_to_array(img) / 255
        img_array_list.append(img_array)
        cls_list.append(1)

X = np.array(img_array_list)
y = np.array(cls_list)

In [4]:
X.shape

(12300, 224, 224, 1)

In [5]:
import sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
import tensorflow.keras.optimizers as opt

model = Sequential()
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 1)))
model.add(MaxPooling2D(pool_size=(8, 8)))
model.add(Dropout(rate=0.5))


model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Dropout(rate=0.5))
model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))

model.add(Dense(units=1, activation='sigmoid'))
Nadam = opt.Nadam(lr=8e-4, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
model.compile(optimizer=Nadam, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 128)     1280      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 27, 27, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 25, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 6, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0

In [9]:
from keras.utils.np_utils import to_categorical
model.fit(X_train, y_train, epochs=8, batch_size=8)

Epoch 1/8
1077/1077 [==============================] - 657s 610ms/step - loss: 0.4625 - accuracy: 0.7495
Epoch 2/8
1077/1077 [==============================] - 646s 600ms/step - loss: 0.4386 - accuracy: 0.7602
Epoch 3/8
1077/1077 [==============================] - 636s 590ms/step - loss: 0.4332 - accuracy: 0.7696
Epoch 4/8
1077/1077 [==============================] - 647s 601ms/step - loss: 0.4001 - accuracy: 0.7914
Epoch 5/8
1077/1077 [==============================] - 646s 599ms/step - loss: 0.3834 - accuracy: 0.7942
Epoch 6/8
1077/1077 [==============================] - 636s 591ms/step - loss: 0.3706 - accuracy: 0.8031
Epoch 7/8
1077/1077 [==============================] - 722s 671ms/step - loss: 0.3635 - accuracy: 0.8082
Epoch 8/8
1077/1077 [==============================] - 668s 620ms/step - loss: 0.3609 - accuracy: 0.8093


In [10]:
predict = model.predict(X_test)[:, 0]
predict

array([0.04437065, 0.999457  , 0.00138694, ..., 0.39331147, 0.29954574,
       0.37756371], dtype=float32)

In [11]:
predict.shape

(3690,)

In [1]:
y_test.shape

NameError: name 'y_test' is not defined

In [14]:
import csv
with open('submission3.csv', 'w', newline='') as writeFile:
    writer = csv.writer(writeFile)
    for p in predict:
        predicts=[]
        predicts.append(p)
        writer.writerow(predicts) 
writeFile.close()

In [ ]:
submit = pd.read_csv('input/1056lab-defect-detection/submission.csv')
submit['defect'] = predict
submit.to_csv('submission.csv', index=False)